# Bio Diversity in National Parks (USA)

### Setting it up 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as pearsonr
import statsmodels as sm

In [ ]:
obs_data = pd.read_csv("observations.csv")
obs_data.head()

In [ ]:
obs_data.describe()
obs_data.info()


In [ ]:
spc_data = pd.read_csv("species_info.csv")
spc_data.conservation_status.unique()


In [ ]:
spc_data.describe()
spc_data.info()

In [ ]:
spc_data.head()

### Understanding variables

In [ ]:
# Conservation Status variable
print(spc_data.conservation_status.value_counts())
print(spc_data.conservation_status.isna().value_counts())

Conservation status variable is just filled with species whose conservation status is of concern. All NaN values in this column are, mainly, from species not falling in any of those categories.

### Cleaning 
Dropping `Nan` and `duplicated` rows in both `spc_data` and `obs_data`

In [ ]:
print(len(spc_data), len(obs_data))
spc_data.drop_duplicates(inplace = True)
# spc_data.dropna(inplace = True) # to be just dropped in columns not conservation_status
obs_data.drop_duplicates(inplace = True)
obs_data.dropna(inplace = True)
print(len(spc_data), len(obs_data))

## Data Overview

### Species Overview
Analysing `spc_data` first. Grasping over `category` distriubution, `conservation_status` against `category` analysis and, for the geeks, average number of `common-names` for each `category`.

In [ ]:

# 1. Category Distribution of Registered Species
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=spc_data.category.value_counts().index, y=spc_data.category.value_counts().values)
ax.set_title("Category Distribution")
ax.set_xlabel("Category")
ax.set_ylabel("Count")
ax.set_yticks(range(0, 5000,500))
plt.show()
plt.clf()

# Some stats about category variable
print("Category with less species is " + str(spc_data.category.value_counts().idxmin()) + " with " + str(spc_data.category.value_counts().min()))
print("Category with more species is " + str(spc_data.category.value_counts().idxmax()) + " with " + str(spc_data.category.value_counts().max()))
print("Average number of species per category is " + str(np.int0(spc_data.category.value_counts().mean())))
spc_no_plants = spc_data[spc_data.category != "Vascular Plant"]
print("Average number of species per category considering Vascular Plants as an outlayer is " + str(np.int0(spc_no_plants.category.value_counts().mean())))





### Conclusion 1:
The most tracked/studied categories are `Vascular Plant` with almost 4500 species within it. The less registered is `Reptile` with 72 species recorded. 

In [ ]:
# 2. Conservation Status against category
# For studying conservation status distribution for each category, rows with NaN values 
# in conservation_stauts column need to be dropped

aux_spc_data = spc_data.dropna()

sns.set(style="whitegrid")
plt.figure(figsize=(10,6))
ax = sns.countplot(x="conservation_status", hue="category", data = aux_spc_data, palette="bright")
ax.set_title('Conservation Status Distribution grouped by Category')
ax.set_xlabel("Conservation Status")
ax.set_ylabel("Count")
ax.legend(title="Category")
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                xytext=(0, 9), 
                textcoords='offset points',
                fontsize=11)
plt.show()

### Observations Overview
Analysing `obs_data`. What is the `total_observations` distribution? 

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10,6))
park_obs = obs_data.groupby('park_name')['observations'].sum().reset_index()

plt.figure(figsize=(12,6))
ax = sns.barplot(x="park_name", y="observations", data=park_obs)
ax.set_yticks(park_obs.observations)
ax.set_yticklabels(park_obs.observations)
plt.tight_layout()
sns.despine()
ax.yaxis.grid(False)
plt.show()

print(park_obs.observations.mean())

print("The average number of observations per National Park is " + str(np.int0(park_obs.observations.mean())))



### Conclusion 2
Yellowstone is, by far, the National Park which has the highest number of observations, Great Smokey Mountains is the one with the fewest.
The average number of observations per national park is 828107.

## Merge Time
Now, we will analyze the most observed categories for each and across all National Parks.

In [ ]:
data = obs_data.merge(spc_data, on="scientific_name")
data.park_name = data.park_name.str.replace(" National Park", "")
data.head()


In [ ]:
data.groupby("park_name")["category"].value_counts().head(40)


In [ ]:
data_obs = data.groupby(["park_name", "category"])["observations"].sum().reset_index()

# Using data_obs to plot the distribution of observations of each category for each National Park

plt.figure(figsize=(18, 8))
sns.set(style="whitegrid")
ax = sns.barplot(x="park_name", y="observations", hue="category", dodge=True, data=data_obs, palette="bright")
plt.title("Observations by National Park", fontsize=20)
plt.xlabel("Park Name", fontsize=12)
plt.ylabel("Observations", fontsize=12)
# ax.set_yticks(range(0,1000000,100000))
# ax.set_yticklabels(range(0,1000000,100000))
# I left the previous lines of code to show how I manually tried to solve this (uncomment to see)
# I finally decided to apply plt.yscale("log") to deal with it
plt.yscale("log")
plt.legend(fontsize=16)
plt.show()
plt.clf()


### Conclusion 3
As expected, `Vascular Plant` is the most tracked category across all National Parks. However, this display of the 4 integrated bar plots makes visible the little differences in between each park for every category. `Amphiabian` and `Reptile` are the less tracked in all parks (as assumable).

### Number of Species of Concern in each National Park (by Category)
Now, we will be plotting a separate graph for each National Park. That way we will be able to see, for each park, the number of  species whose conservation status is beeing tracked clustered by category.

In [ ]:

# Now plotting the distribution of categories of those species whose conservation status is of concern for each park.
park_list = data.park_name.unique()
print(park_list)
for i in park_list:
    aux_data = data[data["park_name"] == i]
    sns.set(style="whitegrid")
    plt.figure(figsize=(12,8))
    plt.title(str(i) + " National Park's Distribution of Conservation Status ", fontsize=17)
    ax = sns.countplot(data=aux_data, x="conservation_status", hue="category")
    ax.legend(title="Category")
    plt.ylabel("Counts")
    plt.xlabel("Conservation Status")
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.0f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                xytext=(0, 9), 
                textcoords='offset points',
                fontsize=11)
    plt.show()
    plt.clf()
   